In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [2]:
pd.set_option('display.max_rows', 200, 'display.max_columns', None)

In [3]:
all_rows = pd.option_context("display.max_rows", None)

In [4]:
months = {m: i for i, m in enumerate("FGHJKMNQUVXZ")}

In [5]:
dailies = pd.read_csv(
    "csi_data.csv.gz", 
    names=[
        "contract", "expiry", "date", 
        "open", "high", "low", "close", 
        "volume", "open_interest",
    ],
    parse_dates=["date"],
)

In [6]:
dailies["expiry_int"] = (
    dailies["expiry"].str.slice(0, 4).astype(int) * 12 + 
    dailies["expiry"].str.slice(4, 5).apply(months.get)
)

In [7]:
dailies["mod_month"] = dailies.eval(
    "expiry_int - (date.dt.year + (expiry_int.mod(12) < date.dt.month.sub(1))) * 12"
)

In [8]:
dailies[["expiry_int", "mod_month"]] = dailies[["expiry_int", "mod_month"]].astype("Int64")

In [9]:
dailies = dailies[dailies.mod_month.between(0, 11)].copy()

In [10]:
dailies["mod_month"] = pd.Categorical.from_codes(dailies["mod_month"], months.keys())

In [11]:
metadata = (
    pd.read_csv("contracts.csv")
    .dropna(subset=["Exch Symbol", "Currency", "Name", "Point Value", 
                    "Symbol", 'Futures Category'])
    .set_index("Symbol")
)

In [12]:
daily_contracts = dailies.contract.drop_duplicates().loc[lambda xs: xs.isin(metadata.index)]

In [13]:
contract_mask = (
    daily_contracts.map(metadata.Currency).isin(["USD", "EUR"])
    &
    ~daily_contracts.isin(["TF", "ER2", "N2U", "SD1", "SD2", "SI4", "F2U"])
    & 
    ~daily_contracts.map(metadata["Point Value"]).str.contains("contract")
    &
    ~daily_contracts.map(metadata["Point Value"]).str.contains("MWH")
    &
    ~daily_contracts.map(metadata["Name"]).str.lower().str.contains("divided by 10")
    &
    ~daily_contracts.map(metadata["Name"]).str.lower().str.contains("last in")
    &
    ~daily_contracts.map(metadata["Name"]).str.lower().str.contains("last trade in close")  
    &
    ~daily_contracts.map(metadata["Name"]).str.lower().str.contains("floor trading only")
    &
    ~daily_contracts.map(metadata["Name"]).str.lower().str.contains("\(floor\)")    
)

In [14]:
multiplier = (
    daily_contracts[contract_mask]
    .map(metadata["Point Value"])
    .fillna("")
    .loc[lambda xs: xs.str.match("^(USD|EUR|\$)") | xs.str.contains("US Dollar")]
    .str.extract("(\d+(?:\.\d+)?)", expand=False).astype(float)
    .rename("multiplier")
)
multiplier = pd.concat([daily_contracts, multiplier], axis=1).dropna().set_index("contract").multiplier
metadata["multiplier"] = multiplier

In [15]:
metadata.dropna(subset=["multiplier"], inplace=True)

In [16]:
dailies = dailies.loc[lambda df: df.contract.isin(metadata.index)]

In [17]:
contract_volumes = (
    dailies
    .assign(decade=lambda df: df.eval("date.dt.year.floordiv(10).mul(10)"))
    .assign(adr=lambda df: df.eval("high - low"))
    .groupby(["contract", "decade"])
    .agg({"volume": "mean", "adr": "mean"})
    .reset_index()
)

In [18]:
contract_volumes["multiplier"] = contract_volumes.contract.map(metadata.multiplier)
contract_volumes["currency"] = contract_volumes.contract.map(metadata.Currency)
contract_volumes["category"] = contract_volumes.contract.map(metadata['Futures Category'])
contract_volumes["exchange"] = contract_volumes.contract.map(metadata.Exchange)
contract_volumes["exch_symbol"] = contract_volumes.contract.map(metadata['Exch Symbol'])
contract_volumes["name"] = contract_volumes.contract.map(metadata['Name'])
contract_volumes["dollar_vol"] = contract_volumes.eval("adr * multiplier * volume")
contract_volumes["cat_rank"] = (
    contract_volumes.groupby(["category", "decade"])
    .dollar_vol.rank(ascending=False)
    .astype(int)
)

In [19]:
shortlist = (
    contract_volumes
    .query("cat_rank <= 5")
    .query("decade == 2010")
    .query("dollar_vol.rank(ascending=False) <= 50")
#         .sort_values(["category", "dollar_vol"], ascending=[True, False])
    .sort_values("dollar_vol", ascending=False)
    [["dollar_vol", "currency", "contract", "exchange", "exch_symbol",
      "category", "cat_rank", "name"]]
    .reset_index(drop=True)
)
with pd.option_context("display.max_rows", None):
    display(
        shortlist
        .style
        .format(subset=["dollar_vol"], formatter="{:.1e}")
        .set_properties(subset=["name"], **{'white-space': 'pre-wrap'})
    )

,dollar_vol,currency,contract,exchange,exch_symbol,category,cat_rank,name
0,4.2e+08,USD,ES,CME,ES,Indexes-U.S.,1,e-mini S&P 500 Index-CME
1,3.2e+08,USD,US,CBT,ZB,Govt Bonds,1,U.S. T-Bond (Floor+Electronic Combined)-CBT
2,1.8e+08,USD,TY,CBT,TYF,Govt Notes,1,10-Year U.S. T-Note (Floor+Electronic Combined)-CBT
3,1.7e+08,EUR,SXE,EUREX,FESX,Indexes-European,1,Euro STOXX 50 Index-EUREX
4,1.2e+08,EUR,FDX,EUREX,FDAX,Indexes-European,2,DAX Index-EUREX
5,1.2e+08,EUR,EBL,EUREX,FGBL,Govt Bonds,2,Euro German Bund-EUREX
6,1.1e+08,USD,CL2,NYMEX,YC,Energy,1,Crude Oil (Floor+Electronic Combined)-NYMEX
7,9.8e+07,USD,FV,CBT,FV,Govt Notes,2,5-Year U.S. T-Note (Floor+Electronic Combined)-CBT
8,9.0e+07,USD,LCO,ICE,B,Energy,2,Brent Crude (Combined)-(IPE)
9,8.7e+07,USD,UL2,CBT,UL,Govt Bonds,3,Ultra T-Bonds (Floor + Electronic Combined)


In [20]:
from matplotlib.backends.backend_pdf import PdfPages

In [21]:
top_ratio = (
    dailies
    .query("date > '2008'")
    .sort_values("volume")
    .groupby(["contract", "date"]).last()
    .groupby(["contract", "mod_month"]).volume.count()
    .groupby("contract").transform(lambda xs: xs / xs.max())
)

In [22]:
with PdfPages('expiry_volumes.pdf') as pdf:
    for contract_rank in tqdm(shortlist.index):

        title = "{contract_rank} - {contract} / {exch_symbol}({exchange}) / {name}".format(
            contract_rank=contract_rank, **shortlist.loc[contract_rank].to_dict()
        )

        plt.figure(dpi=100)

        ax = (
            top_ratio
            .loc[shortlist.contract[contract_rank]]
            .plot.bar(figsize=(8, 5), title=title)
        )
        
        ax.axhline(y=1/12, color="red")
        
        pdf.savefig()
        plt.close()        

  0%|          | 0/50 [00:00<?, ?it/s]

In [202]:
active_months = top_ratio[top_ratio > 1/12].reset_index().drop("volume", axis=1)
dailies = dailies.merge(active_months)
dailies["top_volume"] = (
    dailies
    .groupby(["contract", "date"])
    .volume.rank(ascending=False, method="min") == 1
)
# WARNING: Looks into the future!
dailies["top_volume_future"] = (
    dailies.sort_values("date", ascending=False)
    .groupby(["contract", "expiry"]).top_volume.cummax()
)
live_contracts = dailies[dailies.top_volume_future]

In [203]:
for contract, mult in [("SXE", 1/10), ("CU", 10), ("JY", 10), ("CD", 10)]:
    live_contracts.loc[
        (live_contracts.contract == contract) & 
        (live_contracts.date < "2008"), 
        "close",
    ] *= mult

In [204]:
front = (
    live_contracts.sort_values("expiry")
    .groupby(["contract", "date"], as_index=False).first()
)

In [205]:
rolls = (
    front.loc[lambda df: df.expiry != df.groupby("contract").expiry.shift(-1)]
    .reset_index(drop=True)
)
rolls["spread"] = (
    rolls
    .assign(expiry=lambda df: df.groupby("contract").expiry.shift(-1))
    [["contract", "date", "expiry"]]
    .merge(live_contracts, how="left").close -
    rolls.close
)

rolls = rolls[["contract", "date", "expiry", "spread"]]

front = front.merge(rolls, how="left")
front["spread"].fillna(0, inplace=True)
front["adjustment"] = front.groupby("contract").spread.cumsum()
front = front.set_index(["contract", "date"])

In [206]:
with PdfPages('prices.pdf') as pdf:
    for contract in tqdm(shortlist.itertuples(), total=len(shortlist)):
        title = (f"{contract.Index} - {contract.contract} / "
                 f"{contract.exch_symbol}({contract.exchange}) / {contract.name}")
        plt.figure(dpi=100)

        (
            front
            .loc[contract.contract]
#             ["2007-10":"2008-03"]
            .eval("close + adjustment")
            .plot(figsize=(8, 5), title=title)
        )     
        
        pdf.savefig()
        plt.close()        

  0%|          | 0/50 [00:00<?, ?it/s]

In [36]:
# from bqplot import pyplot as plt
# from bqplot import OrdinalScale
# df_subset = df.tail(180)
# fig = plt.figure()
# plt.scales(scales={'x': OrdinalScale()})
# axes_options = {'x': {'label': 'Date', 'tick_format': '%Y-%m-%d', 
#                       'num_ticks': 8, "tick_rotate": 45, 'tick_style': {'text-anchor': 'start'}}, 
#                 'y': {'label': 'Price', 'label_offset': "3em", 'tick_format': '.0f'}}
# ohlc = plt.ohlc(
#     x=df_subset.index,
#     y=df_subset[ohlc_cols],
#     axes_options=axes_options,
#     marker="candle", 
#     stroke="black",
# )
# ohlc.colors=["lime", "tomato"]
# fig